# Automatic (derivatives) for the people and a song of ice and fire

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math


### Exercise 1

#### Part 1

We import sys and get float info

In [1]:
import sys
print(sys.float_info)

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)


Max is the largest representable number in computers, similarily min is the smallest representable number in computers.  
Epsilon tells us the smallest number we can add to one to get something other than one.

Most modern computers work on a finite 64-bit format and processes 64 bits at a time. The values given from `sys.float_info` come from the IEEE 754 standard**REFERENCE**. In this exercise we will focus on IEEE 754 double precision (64-bit) floating-point number.

From the 64 bits we have IEEE reserves one sign bit, 11 exponent bits and 52 fraction(mantissa) bits. To also represent numbers that have absoloute value less than one a number *bias* is added to the exponent. Bias is a fixed number and in 64 bits it is roughly half the exponent; $2^{11-1}-1 = 1023$. With this we can make a formula: 

\begin{equation}
±q2^{E-e}
\end{equation}

where ± is the sign bit, q the mantissa, E the exponent and $e$ the bias. E can have values from 0 to $2^{11} - 1$ = 2047. There are two special numbers to represent zero and infinity which are respectively $E = 0$, and $E = 2047$. With bias this gives us the largest possible exponent $2046 - 1023 = 1023$ and smallest exponent $1 - 1023 = -1022$  

To get the max value the exponent needs to be 1023, and the fraction needs be all ones
